Talk about the data and what we are going to do this with data

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df = pd.read_excel("../data/raw/litigation_cases.xlsx", skiprows=5, skipfooter=7)
df.head()

,LIT Leave Decision Date - Year,Country of Citizenship,LIT Leave Decision Desc,LIT Case Type Group Desc,LIT Case Type Desc,LIT Filed By Desc,LIT Tribunal Type Desc,LIT Primary Office Type Desc,LIT Primary Office Regional Group Desc,LIT Litigation Count
0,2023,India,Dismissed at Leave,RAD Decisions,RAD-Dismissed-Confirmed-Same,Person Concerned,Federal Court,Inland,IRCC Vancouver Adm,12
1,2021,Fiji,Allowed,IAD Decisions,IAD-Removal Order,Person Concerned,Federal Court,Port of Entry,Pacific Highway District,1
2,2023,Russia,Discontinued - Withdrawn at Leave,Visa Officer Refusal,Visa Officer Decision,Person Concerned,Federal Court,International Network,Unspecified,7
3,2023,Republic of Indonesia,Discontinued - Consent at Leave,HC Decisions,HC Refusal-Perm Res,Person Concerned,Federal Court,Inland,Humanitarian Migration and Integrity Division,2
4,2018,Georgia,Discontinued - Consent at Leave,Visa Officer Refusal,Visa Officer Decision,Person Concerned,Federal Court,International Network,Unspecified,2


In [9]:
print(df["LIT Case Type Group Desc"].value_counts())
print("\n", df["LIT Leave Decision Desc"].value_counts())

LIT Case Type Group Desc
RAD Decisions           3403
Visa Officer Refusal    2394
Removal Order           1965
RPD Decisions           1741
HC Decisions            1091
PRRA                     963
Mandamus                 828
Removal Arrangements     568
IAD Decisions            441
ID Decisions             176
Other                     88
Refugee Eligibility       41
In-Canada Classes         35
Detention                 24
Citizenship Program       20
DFN                        2
Danger Opinions            1
Name: count, dtype: int64

 LIT Leave Decision Desc
Dismissed at Leave                   6011
Allowed                              3148
Discontinued - Withdrawn at Leave    2861
Discontinued - Consent at Leave      1454
Allowed - Consent                     260
Not Started at Leave                   37
No Leave Required                       8
Leave Exception                         2
Name: count, dtype: int64


We have grouped RAD decisions, RPD decisions, PRRA and Refugee Eligibility decisions as litigation applications which are related to refugee cases. We have done this because our partner organization were more intereseted in non-refugee cases as they dealt with those in their day to day.
We have also  standardrized "Discontinued - Withdrawn at Leave" and "Discontinued - Consent at Leave" to Discontinued and "Allowed - Consent" to just allowed. We are doing this because there is only just a small difference between this types and we want to see comparison with dismissed, allowed and discontinued cases as a whole.

In [14]:
# Standardizing Leave decision
df['LIT Leave Decision Desc'] = df['LIT Leave Decision Desc'].replace(
    to_replace=r'^Discontinued.*', value='Discontinued', regex=True
)

df['LIT Leave Decision Desc'] = df['LIT Leave Decision Desc'].replace(
    to_replace=r'^Dismissed.*', value='Dismissed', regex=True
)

df['LIT Leave Decision Desc'] = df['LIT Leave Decision Desc'].replace(
    to_replace=r'^Allowed.*', value='Allowed', regex=True
)

In [ ]:
# Grouping refugee case type
refugee_case_types = [
    "RAD Decisions",
    "RPD Decisions",
    "PRRA",
    "Refugee Eligibility"
]

df_refugee = df[df["LIT Case Type Group Desc"].isin(refugee_case_types)]
df_non_refugee = df[~df["LIT Case Type Group Desc"].isin(refugee_case_types)]

In [31]:
import pandas as pd
import plotly.express as px

# Prepare data as before
df_refugee = df_refugee.copy()
df_non_refugee = df_non_refugee.copy()
df_refugee['Case Type'] = 'Refugee'
df_non_refugee['Case Type'] = 'Non-Refugee'

df_combined = pd.concat([df_refugee, df_non_refugee])

lit_counts = (
    df_combined.groupby(["Country of Citizenship", "Case Type"])["LIT Litigation Count"]
    .sum()
    .reset_index()
)

total_counts = (
    lit_counts.groupby("Country of Citizenship")["LIT Litigation Count"]
    .sum()
    .reset_index()
)

# Use nlargest to get top 10 countries by total litigation count
top_countries = total_counts.nlargest(10, "LIT Litigation Count")
ordered_countries = top_countries["Country of Citizenship"].tolist()

# Filter data to include only these countries
filtered_counts = lit_counts[lit_counts["Country of Citizenship"].isin(ordered_countries)]

# Plot stacked horizontal bar chart
fig = px.bar(
    filtered_counts,
    x="LIT Litigation Count",
    y="Country of Citizenship",
    color="Case Type",
    text='LIT Litigation Count',
    labels={"LIT Litigation Count": "Litigation Count", "Country of Citizenship": "Country"},
    barmode='stack',
    category_orders={"Country of Citizenship": ordered_countries},  # Enforce order here
    orientation='h'
)

fig.update_traces(texttemplate='%{text:.0f}', textposition='inside', cliponaxis=False)

# Add total litigation count as annotations
for i, row in top_countries.iterrows():
    fig.add_annotation(
        x=row["LIT Litigation Count"] + (row["LIT Litigation Count"] * 0.02),  # Slightly offset right
        y=row["Country of Citizenship"],
        text=f"Total: {int(row['LIT Litigation Count'])}",
        showarrow=False,
        font=dict(size=14, color="black"),
        xanchor='left',
        yanchor='middle'
    )

fig.update_layout(
    xaxis_title="Litigation Count",
    yaxis_title="Country of Citizenship",
    legend_title_text="Case Type",
    font=dict(size=16),
    margin=dict(l=160, r=40, t=100, b=120),
    height=600,
    width=1000
)

fig.show()

Nigeria, Mexico, Haiti, Columbia and USA have majority of cases related to refugee,
Almost all the cases of Philippines and Iran are related to non refugee,
India have majority of cases in non-refugee but also have the 2nd highest cases of refugee
Pakistan and China have an similar number of refugee and non-refugee case.

In [38]:
time_series = (
    df_combined.groupby(["LIT Leave Decision Date - Year", "Case Type"])["LIT Litigation Count"]
    .sum()
    .reset_index()
    .rename(columns={"LIT Leave Decision Date - Year": "Year"})
)

# Plot time series
fig_time = px.line(
    time_series,
    x="Year",
    y="LIT Litigation Count",
    color="Case Type",
    markers=True,
    labels={"LIT Litigation Count": "Litigation Count", "Year": "Year"},
)

fig_time.update_layout(
    font=dict(size=16),
    margin=dict(l=80, r=40, t=100, b=80),
    height=600,
    width=900
)

fig_time.show()

We see same trend from 2018 to 2020, after 2020 there is an increasing for non-refugee litigation application, but for refugee-related litigation application we see an increase for 2021 but then a decrease afterwards.

Now lets see if there is some difference in desicion outcome percentage between refugee related cases and non refugee related cases across different years.

In [45]:
# Step 1: Compute top 3 most frequent decisions overall
top_decisions = (
    df_combined.groupby("LIT Leave Decision Desc")["LIT Litigation Count"]
    .sum()
    .nlargest(3)
    .index
    .tolist()
)

# Step 2: Filter for only those top 3 decision descriptions
filtered_df = df_combined[df_combined["LIT Leave Decision Desc"].isin(top_decisions)]

# Step 3: Group by Year, Case Type, and Decision Description
grouped = (
    filtered_df.groupby(["LIT Leave Decision Date - Year", "Case Type", "LIT Leave Decision Desc"])["LIT Litigation Count"]
    .sum()
    .reset_index()
)

# Step 4: Calculate percentages within each Year + Case Type
grouped["Total"] = grouped.groupby(["LIT Leave Decision Date - Year", "Case Type"])["LIT Litigation Count"].transform("sum")
grouped["Percentage"] = (grouped["LIT Litigation Count"] / grouped["Total"]) * 100

# Step 5: Plot area chart (visually like a line chart)
fig = px.area(
    grouped,
    x="LIT Leave Decision Date - Year",
    y="Percentage",
    color="LIT Leave Decision Desc",
    facet_col="Case Type",
    line_group="LIT Leave Decision Desc",
    labels={
        "Percentage": "Decision Percentage",
        "LIT Leave Decision Date - Year": "Year",
        "LIT Leave Decision Desc": "Decision Type"
    },
    facet_col_spacing=0.15  # ✅ Correct placement here
)

fig.update_traces(stackgroup=None)

fig.update_layout(
    font=dict(size=14),
    height=600,
    width=1100,
    legend_title_text="Decision Outcome",
    margin=dict(t=80, b=60, l=60, r=40)
)

fig.show()


- Percentage of litigation cases being dismissed is much higher for refugee cases as compared to non refugee cases and percentage of discontinued cases for non-refugee is very much higher as compared to refugee cases.
- the percentage of litigation application which were allowed range between 20-30 percentage for refugee cases, but for non-refugee cases it is somewhere between 10-20 percentage

- Before 2020, which was covid, we see that the number of litigation which were discontinued were just arounf 30% and those which were dismissed were arounf 45%, but during and then after covid we see that higher percentage of litigation cases being discontinued as compared to dismissed cases. 

- we see that the dismissed percentage dipping to around 60 % after covide but we see it increasing to near 70% in 2023
- Allowed percentage for both refugee and non-refugee decrease in covid and then increase in 2021 then decrease in 2023
- we see increase in litigation count dicisions being discontinued in 2020, probably due to covid.


Comparing with previous graph of time series of refugee and non-refugee litigation count, we can see that the number of litigation count does not have much effect with the decision percentage as for both we see some what similar trend for their decision pecentage.

In [ ]:
period_map = {
    2018: 'Pre-COVID',
    2019: 'Pre-COVID',
    2020: 'COVID',
    2021: 'COVID',
    2022: 'Post-COVID',
    2023: 'Post-COVID'
}
df_combined['Period'] = df_combined['LIT Leave Decision Date - Year'].map(period_map)

# Compute total counts and get top 4 countries
total_counts = (
    df_combined.groupby("Country of Citizenship")["LIT Litigation Count"]
    .sum()
    .nlargest(4)
    .index
)

# Filter for top 4 countries
top_4_df = df_combined[df_combined["Country of Citizenship"].isin(total_counts)]

# Step 4: Group by Country and Period
grouped = (
    top_4_df.groupby(["Country of Citizenship", "Period"])["LIT Litigation Count"]
    .sum()
    .reset_index()
)

# Compute total per country for ordering
country_order = (
    grouped.groupby("Country of Citizenship")["LIT Litigation Count"]
    .sum()
    .sort_values(ascending=False)
    .index
    .tolist()
)

fig = px.bar(
    grouped,
    x="Country of Citizenship",
    y="LIT Litigation Count",
    color="Period",
    barmode="group",
    category_orders={
        "Period": ["Pre-COVID", "COVID", "Post-COVID"],
        "Country of Citizenship": country_order  # enforce ascending order
    },
    labels={"LIT Litigation Count": "Litigation Count"}
)

fig.update_layout(
    xaxis_title="Country of Citizenship",
    yaxis_title="Total Litigation Count",
    font=dict(size=16),
    height=600,
    width=1000,
    legend_title_text="Period"
)

fig.show()

In [57]:
import plotly.express as px
import pandas as pd

# Step 1: Get top 4 countries by total litigation count
top_countries = (
    df_combined.groupby("Country of Citizenship")["LIT Litigation Count"]
    .sum()
    .nlargest(4)
    .index
    .tolist()
)

# Step 2: Filter data for these countries
top_df = df_combined[df_combined["Country of Citizenship"].isin(top_countries)]

# Step 3: Get top 3 decision descriptions overall within these countries
top_decisions = (
    top_df.groupby("LIT Leave Decision Desc")["LIT Litigation Count"]
    .sum()
    .nlargest(3)
    .index
    .tolist()
)

# Step 4: Filter for those decisions
filtered_df = top_df[top_df["LIT Leave Decision Desc"].isin(top_decisions)]

# Step 5: Group by Year, Country, and Decision
grouped = (
    filtered_df.groupby(["LIT Leave Decision Date - Year", "Country of Citizenship", "LIT Leave Decision Desc"])["LIT Litigation Count"]
    .sum()
    .reset_index()
)

# Step 6: Calculate percentage within each Year + Country
grouped["Total"] = grouped.groupby(["LIT Leave Decision Date - Year", "Country of Citizenship"])["LIT Litigation Count"].transform("sum")
grouped["Percentage"] = (grouped["LIT Litigation Count"] / grouped["Total"]) * 100

# Step 7: Plot
fig = px.area(
    grouped,
    x="LIT Leave Decision Date - Year",
    y="Percentage",
    color="LIT Leave Decision Desc",
    facet_col="Country of Citizenship",
    facet_col_wrap=2,
    line_group="LIT Leave Decision Desc",
    labels={
        "Percentage": "Decision Percentage",
        "LIT Leave Decision Date - Year": "Year",
        "LIT Leave Decision Desc": "Decision Type"
    },
)

# Make lines (not stacked)
fig.update_traces(stackgroup=None)

fig.update_layout(
    font=dict(size=14),
    height=700,
    width=1200,
    legend_title_text="Decision Outcome",
    margin=dict(t=80, b=60, l=60, r=40)
)

for i in range(1, len(top_countries) + 1):
    fig.layout[f'yaxis{i}'].dtick = 10

fig.show()